In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime

start_date = datetime(2015, 1, 1, 1, 1)
end_date = datetime(2020, 1, 1, 1, 1)
''' 
nvda = yf.download("NVDA", start="2015-01-01", end="2020-01-01")
nvda.to_csv("nvda_2015_2020.csv")
ticker = yf.Ticker('NVDA')
ticker.get_earnings_dates(limit=100).to_csv('nvda_earnings_dates')
'''

' \nnvda = yf.download("NVDA", start="2015-01-01", end="2020-01-01")\nnvda.to_csv("nvda_2015_2020.csv")\nticker = yf.Ticker(\'NVDA\')\nticker.get_earnings_dates(limit=100).to_csv(\'nvda_earnings_dates\')\n'

In [3]:
prices = pd.read_csv('nvda_2015_2020.csv').iloc[2:,:].rename(columns={'Price': 'Date'})
prices['Date'] = pd.to_datetime(prices['Date'])
prices = prices.set_index('Date')
prices = prices.astype(float)
prices

,Close,High,Low,Open,Volume
Date,,,,,
2015-01-02,0.483099,0.486699,0.475419,0.483099,113680000.0
2015-01-05,0.474939,0.484539,0.472780,0.483099,197952000.0
2015-01-06,0.460540,0.476139,0.460060,0.475659,197764000.0
2015-01-07,0.459340,0.467980,0.457900,0.463900,321808000.0
2015-01-08,0.476619,0.479499,0.464380,0.464620,283780000.0
...,...,...,...,...,...
2019-12-24,5.940049,5.957226,5.895241,5.929594,138864000.0
2019-12-26,5.954238,5.982368,5.924615,5.943534,182852000.0
2019-12-27,5.896485,5.982865,5.870098,5.969422,254644000.0


In [4]:
earnings = pd.read_csv('nvda_earnings_dates')
# fix formatting with dates, remove time zone and hours/mins
earnings['Earnings Date'] = pd.to_datetime(earnings['Earnings Date'], utc=True)
earnings['Earnings Date'] = earnings['Earnings Date'].dt.tz_localize(None)
earnings['Earnings Date'] = pd.to_datetime(earnings['Earnings Date'].dt.date)
# filter earnings dates to start and end date
earnings = earnings[(start_date <= earnings['Earnings Date']) & (earnings['Earnings Date'] <= end_date)]
# remove the first earnings date (not enough stock price data)
earnings = earnings.drop(earnings.index[-1])
earnings = earnings.set_index('Earnings Date')
earnings

,EPS Estimate,Reported EPS,Surprise(%)
Earnings Date,,,
2019-11-14,0.39,0.45,13.07
2019-08-15,0.29,0.31,8.29
2019-05-16,0.20,0.22,8.69
2019-02-14,0.19,0.20,6.48
2018-11-15,0.47,0.46,-1.32
2018-08-16,0.44,0.49,9.80
2018-05-10,0.40,0.51,28.67
2018-02-08,0.29,0.39,34.37
2017-11-09,0.24,0.33,40.83


In [25]:
import numpy as np
price_data = pd.DataFrame()
volume_data = pd.DataFrame()

for date in earnings.index:
    idx = prices.index.get_loc(date)
    
    earnings_day_price = prices['Close'].iloc[idx]
    price_data[date] = ((prices['Close'].iloc[idx-60:idx+2] - earnings_day_price) / earnings_day_price).reset_index(drop=True)
    
    volume_data[date] = pd.Series([prices['Volume'].iloc[idx-60:idx].mean(), prices['Volume'].iloc[idx-7:idx].mean()], index=['avg volume 60', 'avg volume 7'])
    
earnings_data = price_data.set_index(np.arange(60, -2, -1))
earnings_data = earnings_data.T
volume_data = volume_data.T
earnings_data['MONEY'] = earnings_data[-1] >= earnings_data[0] * 1.03
#earnings_data
volume_data.join(earnings_data)

,avg volume 60,avg volume 7,60,59,58,57,56,55,54,53,...,7,6,5,4,3,2,1,0,-1,MONEY
2019-11-14,3.230485e+08,2.666354e+08,-0.184610,-0.183419,-0.226468,-0.212134,-0.229515,-0.231517,-0.203966,-0.201535,...,-0.000858,-0.010296,-0.006912,-0.009581,-0.007674,-0.000858,-0.005815,0.0,-0.026693,False
2019-08-15,3.988891e+08,3.479977e+08,0.041092,0.021889,-0.010876,-0.025445,-0.037665,-0.057740,-0.064932,-0.089467,...,0.024064,0.034415,0.063790,0.036365,0.018014,0.048935,0.008738,0.0,0.072528,True
2019-05-16,5.043915e+08,5.554640e+08,-0.011257,-0.028593,-0.007266,-0.010384,-0.020300,-0.030839,-0.037019,-0.023348,...,0.080654,0.085710,0.062426,0.053873,-0.010862,0.011548,-0.003808,0.0,-0.022848,False
2019-02-14,7.629827e+08,5.764606e+08,0.308404,0.063001,-0.064548,-0.036232,-0.064484,-0.062609,-0.010567,-0.006171,...,-0.029638,-0.009901,-0.046010,-0.041157,-0.052288,-0.021743,-0.010677,0.0,0.018184,True
2018-11-15,4.711557e+08,4.994034e+08,0.297872,0.317724,0.344292,0.362464,0.354958,0.376007,0.372647,0.386828,...,0.042838,0.056327,0.017787,0.016206,-0.063491,-0.015218,-0.025693,0.0,-0.187559,False
2018-08-16,3.570693e+08,3.447817e+08,-0.057839,-0.038456,-0.037873,-0.031697,-0.034377,-0.017285,-0.020393,0.000699,...,-0.001903,0.003807,-0.003807,-0.010294,-0.005127,0.015499,0.006370,0.0,-0.049021,False
2018-05-10,5.850401e+08,4.509760e+08,-0.106272,-0.072502,-0.052986,-0.063205,-0.043074,-0.072156,-0.069120,-0.054588,...,-0.126821,-0.130012,-0.104332,-0.080998,-0.044017,-0.037404,-0.016722,0.0,-0.021528,False
2018-02-08,5.700372e+08,7.671474e+08,-0.007035,-0.023160,-0.016039,-0.035334,-0.027846,-0.028994,-0.016498,-0.007448,...,0.115851,0.130011,0.105645,0.073556,-0.017562,0.037054,0.051857,0.0,0.066936,True
2017-11-09,5.720610e+08,4.215806e+08,-0.196353,-0.214261,-0.214115,-0.225550,-0.208309,-0.192480,-0.195451,-0.202172,...,0.007257,0.009156,0.003019,0.016414,0.020992,0.032681,0.018703,0.0,0.052698,True
2017-08-10,8.841012e+08,5.635537e+08,-0.170414,-0.225534,-0.193092,-0.174457,-0.156854,-0.168205,-0.158857,-0.160739,...,-0.001518,-0.002125,0.010562,0.014993,0.046194,0.033749,0.044737,0.0,-0.053296,False


Close    float64
dtype: object

DatetimeIndex(['2015-01-02', '2015-01-05', '2015-01-06', '2015-01-07',
               '2015-01-08', '2015-01-09', '2015-01-12', '2015-01-13',
               '2015-01-14', '2015-01-15',
               ...
               '2019-12-17', '2019-12-18', '2019-12-19', '2019-12-20',
               '2019-12-23', '2019-12-24', '2019-12-26', '2019-12-27',
               '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', name='Date', length=1258, freq=None)